In [ ]:
from smile.common import *
from smile.scale import scale as s
from smile.startup import InputSubject
from smile.math_distract import MathDistract
​
from util import proc_subjid
​
# config section
font_size = 75
resp_keys = ['F', 'J']
​
resp_map_p = {'target': 'F', 'lure': 'J'}
resp_map_c = {'lure': 'F', 'target': 'J'}
​
ISI_dur = 0.5
ISI_jitter = 0.5
STUDY_TIME = 1.5
MATHDURATION = 20
MATHVARIABLES = 3
​
inst_title_font_size = 40
inst_font_size = 25
​
inst_text = """[u][size={}]WORD RECOGNITION INSTRUCTIONS[/size][/u]
​
In this task, you will be studying a list of items and then be asked to discriminate between items that you studied (targets) and items you did not study (lures).
​
In between each study and test phase, you will be asked to do 30 seconds of math problems.
​
During the study phase, pay careful attention to each item as it is on the screen.
​
During the test phase, you will press {} if the item you are viewing is one that was on the study list, and press {} if the item is new and hadn't been studied previously. You will be reminded of these keys before each test phase.
​
Please answer as quickly and as accurately as possible during the experiment.
    
Press the ENTER key to continue."""
​
recall_instructions = """REMEMBER: 
For a old word you studied press: {}
​
For a new word you did not study press: {}
​
Press the ENTER key to start the TEST.
"""
​
math_instructions = """You are about to see a series of math problems.
​
If the problem is correct (e.g., 1+4=5) press: {}
​
If the problem is incorrect (e.g., 1-9=-7) press: {}
​
Press the ENTER key to start the MATH.
"""
​
@Subroutine
def Instruct(self, resp_map):
    # show the instructions
    Label(text=Ref(inst_text.format, Ref(int, s(inst_title_font_size)),
                   resp_map['target'], resp_map['lure']), 
          font_size=s(inst_font_size),
          text_size=(self.exp.screen.width*0.75, None),
          markup=True)
    with UntilDone():
        KeyPress(keys=['ENTER'])
​
@Subroutine
def StudyTrial(self, block_num, trial_num, cur_trial):
    # present the stimulus
    stim = Label(text=cur_trial['description'],
                 font_size=s(font_size),
                 duration=STUDY_TIME)
    # wait the ISI with jitter
    Wait(ISI_dur, jitter=ISI_jitter)
​
    # log the result of the trial
    Log(name='word_study', 
        log_dict=cur_trial,
        block_num=block_num,
        trial_num=trial_num)
​
@Subroutine
def TestTrial(self, block_num, trial_num, cur_trial, resp_map):
    # present the stimulus
    stim = Label(text=cur_trial['description'],
                 font_size=s(font_size))
    with UntilDone():
        # make sure the stimulus has appeared on the screen
        Wait(until=stim.appear_time)
        
        # collect a response (with no timeout)
        kp = KeyPress(keys=resp_keys, 
                      base_time=stim.appear_time['time'],
                      correct_resp=resp_map[cur_trial['novelty']])
    # wait the ISI with jitter
    Wait(ISI_dur, jitter=ISI_jitter)
​
    # log the result of the trial
    Log(name='word_test', 
        resp_map=resp_map,
        log_dict=cur_trial,
        block_num=block_num,
        trial_num=trial_num,
        stim_on=stim.appear_time,
        resp=kp.pressed,
        resp_time=kp.press_time,
        rt=kp.rt,
        correct=kp.correct,
       )
​
​
@Subroutine
def WordExp(self, blocks):
    count_balance = Func(proc_subjid, Ref.getattr(self.exp, "subject")).result
    with If(count_balance):
        self.resp_map_word = resp_map_c
    with Else():
        self.resp_map_word = resp_map_p
        
    # show the instructions
    Instruct(self.resp_map_word)
    Wait(0.5)
​
    # loop over the blocks
    with Loop(blocks) as block:
        # make sure they are ready to continue
        Label(text='Press the ENTER key to\nstart the next STUDY block.', 
              font_size=s(inst_title_font_size), halign='center')
        with UntilDone():
            KeyPress(keys=['ENTER'])
​
        # add in some delay before the start of the block
        Wait(ISI_dur*2, jitter=ISI_jitter)
        
        # loop over the trials
        with Loop(block.current['study']) as trial:
            StudyTrial(block.i, trial.i, trial.current)
            
        # MATH INSTRUCTIONS
        Label(text=Ref(math_instructions.format,
                       self.resp_map_word['target'], 
                       self.resp_map_word['lure']), 
              font_size=s(inst_font_size), halign='center')
        with UntilDone():
            KeyPress(keys=['ENTER'])
​
        # MATH Time
        MathDistract(duration=MATHDURATION,
                     keys={'True':self.resp_map_word['target'], 
                           'False':self.resp_map_word['lure']},
                     num_vars=MATHVARIABLES)
        
        # make sure they are ready to continue
        Label(text=Ref(recall_instructions.format,
                       self.resp_map_word['target'], 
                       self.resp_map_word['lure']), 
              font_size=s(inst_font_size), halign='center')
        with UntilDone():
            KeyPress(keys=['ENTER'])
        # loop over the trials
        with Loop(block.current['test']) as trial:
            TestTrial(block.i, trial.i, trial.current,
                      self.resp_map_word)
